In [38]:
import json
from tqdm import tqdm
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from gensim.models import Word2Vec
import numpy as np
import torch.nn as nn
import torch
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA

In [2]:
data_df = pd.read_json("combined_book_data.json")
print(data_df.head())

            isbn                                             title  \
0  9784297106614                     ゼロからわかるAmazon Web Services超入門   
1  9784297108892  図解即戦力 Amazon Web Servicesのしくみと技術がこれ1冊でしっかりわかる教科書   
2  9784798162560                                    AWSクラウドの基本と仕組み   
3  9784296202041         Amazon Web Services基礎からのネットワーク&サーバー構築改訂4版   
4  9784797392562                Amazon Web Services 業務システム設計・移行ガイド   

                          authors  \
0                          [大澤文孝]   
1                         [小笠原種高]   
2                  [亀田 治伸, 山田 裕進]   
3         [大澤文孝, 玉川憲, 片山暁雄, 今井雄太]   
4  [佐々木拓郎, 林晋一郎, 瀬戸島敏宏, 宮川亮, 金澤圭]   

                                         description   categories  
0       AWSでWebサイトを構築・運用!実際に動かして学べる1冊。練習問題&解答・解説集付き。         None  
1  Amazon Web Servicesのしくみがキーワードベースでわかる!ネットワーク&クラ...         None  
2  AWS入門書の決定版 アマゾン ウェブ サービス(AWS)社のプロダクトマーケティング エバ...         None  
3  AWSを使ってインフラの基本を学べる本。「クラウドを触って学ぶ」コンセプトが広く受け入れられ...         None  
4  社内システムのクラウド化を

In [6]:
all_categories = data_df['categories'].explode().unique()
print(len(all_categories))
print(all_categories)

7
[None 'Computers' 'Technology & Engineering' 'Reference'
 'Business & Economics' 'Language Arts & Disciplines' 'Self-Help']


In [7]:
all_authors = data_df['authors'].explode().unique()
print(len(all_authors))
print(all_authors)

145
['大澤文孝' '小笠原種高' '亀田 治伸' '山田 裕進' '玉川憲' '片山暁雄' '今井雄太' '佐々木拓郎' '林晋一郎' '瀬戸島敏宏'
 '宮川亮' '金澤圭' '矢沢久雄' '柴田望洋' '増田智明' 'WINGSプロジェクト' '片渕彼富' 'グレゴリー サティア'
 'ダウグブラウン' 'Bjarne Stroustrup' '信男·斎藤' '中山 清喬' '国本 大悟' '株式会社フレアリンク' '高橋麻奈'
 '谷本 心' 'リブロワークス' '佐々木整' '石井 真' '株式会社カサレアル' 'きしだなおき' '山本裕介' '杉山貴章' '三谷 純'
 '近藤 雄太' '正野 裕大' '坂井 潔' '鳥越 淳' '笠原 辰仁' '勝俣智成' '佐伯昌樹' '原田登志' '西潤史郎' 'ミック'
 '常松 祐一' '安達裕哉' 'ウォルター・アイザックソン' 'Mana' '佐竹陽一' '山﨑翔平' '小倉大' '峯侑資' '馬田隆明'
 '山田祥寛' '西岡杏' '増井 敏克' 'IPUSIRON' 'トム・ロング (ソフトウェア工学)' '高松智史' '木下勝寿'
 'アレックス・シュウ' '神林飛志' '荒川傑' '菱田真人' 'シド・ハレル' 'azu' 'Suguru Inatomi' '荒木俊哉'
 'アンドリュー・スチュワート' 'ピョートル・フェリクス・グジバチ' '木暮太一' 'Will Larson' 'フレアリンク' '牛尾剛'
 'Tomasz Lelek' 'Jon Skeet' '山田育矢' '鈴木正敏' '李凌寒' '岡野原大輔' '武内覚'
 'Vladimir Khorikov' '大塚亜周' '稲葉志奈' '金森悠' 'samemaru' '圓山伊吹' '植田将基' '関口裕'
 'イーサン・クロス' 'ChatGPTビジネス研究会' '田中ひさてる' '賀好昭仁' '石川明' '安藤広大' 'リチャード・セイラー'
 'キャス・サンスティーン' '望月俊孝' 'ダリル・ブリッカー' 'ジョン・イビットソン' 'Michael Hausenblas' '森 巧尚'
 '寺田 学' '辻 真吾' '鈴木 たかのり' '福島 真太朗' 'クジラ飛行机' '立山秀利' 'ブルース・シュナイアー' '菊地正

In [8]:
null_rows = data_df[data_df.isnull().any(axis=1)]
print(null_rows)

             isbn                                             title  \
0   9784297106614                     ゼロからわかるAmazon Web Services超入門   
1   9784297108892  図解即戦力 Amazon Web Servicesのしくみと技術がこれ1冊でしっかりわかる教科書   
2   9784798162560                                    AWSクラウドの基本と仕組み   
3   9784296202041         Amazon Web Services基礎からのネットワーク&サーバー構築改訂4版   
5   9784844381679                       Amazon Web Servicesネットワーク入門   
..            ...                                               ...   
95  9784863543324                              Vue.jsビギナーズガイド 3.x対応   
96  9784863543522                            現場で使えるVue.js 3.X 実践ガイド   
97  9784297130725                               Vue 3 フロントエンド開発の教科書   
98  9784798158921                                 動かして学ぶ！Vue.js開発入門   
99  9784844398356                       Vue.jsとFirebaseで作るミニWebサービス   

                    authors  \
0                    [大澤文孝]   
1                   [小笠原種高]   
2            [亀田 治伸, 山田 裕進]   
3   [大澤文孝, 玉川憲, 片山暁雄, 今

In [9]:
data_df['authors'] = data_df['authors'].apply(lambda x: x if isinstance(x, list) and x else ["Unknown"])
data_df['description'] = data_df['description'].fillna("Unknown")
data_df['categories'] = data_df['categories'].apply(lambda x: x if isinstance(x, list) and x else ["Unknown"])

In [10]:
print(data_df.head())

            isbn                                             title  \
0  9784297106614                     ゼロからわかるAmazon Web Services超入門   
1  9784297108892  図解即戦力 Amazon Web Servicesのしくみと技術がこれ1冊でしっかりわかる教科書   
2  9784798162560                                    AWSクラウドの基本と仕組み   
3  9784296202041         Amazon Web Services基礎からのネットワーク&サーバー構築改訂4版   
4  9784797392562                Amazon Web Services 業務システム設計・移行ガイド   

                          authors  \
0                          [大澤文孝]   
1                         [小笠原種高]   
2                  [亀田 治伸, 山田 裕進]   
3         [大澤文孝, 玉川憲, 片山暁雄, 今井雄太]   
4  [佐々木拓郎, 林晋一郎, 瀬戸島敏宏, 宮川亮, 金澤圭]   

                                         description   categories  
0       AWSでWebサイトを構築・運用!実際に動かして学べる1冊。練習問題&解答・解説集付き。    [Unknown]  
1  Amazon Web Servicesのしくみがキーワードベースでわかる!ネットワーク&クラ...    [Unknown]  
2  AWS入門書の決定版 アマゾン ウェブ サービス(AWS)社のプロダクトマーケティング エバ...    [Unknown]  
3  AWSを使ってインフラの基本を学べる本。「クラウドを触って学ぶ」コンセプトが広く受け入れられ...    [Unknown]  
4  社内システムのクラウド化を

In [28]:
unique_authors = pd.Series([author for sublist in data_df['authors'] for author in sublist]).unique()
author2id = {author: i for i, author in enumerate(unique_authors)}
print(author2id)
author_id_df = pd.DataFrame(list(author2id.items()), columns=['name', 'id'])
author_id_df.to_csv('author2id.csv', index=False)

{'大澤文孝': 0, '小笠原種高': 1, '亀田 治伸': 2, '山田 裕進': 3, '玉川憲': 4, '片山暁雄': 5, '今井雄太': 6, '佐々木拓郎': 7, '林晋一郎': 8, '瀬戸島敏宏': 9, '宮川亮': 10, '金澤圭': 11, '矢沢久雄': 12, '柴田望洋': 13, '増田智明': 14, 'WINGSプロジェクト': 15, '片渕彼富': 16, 'グレゴリー サティア': 17, 'ダウグブラウン': 18, 'Bjarne Stroustrup': 19, '信男·斎藤': 20, '中山 清喬': 21, '国本 大悟': 22, '株式会社フレアリンク': 23, '高橋麻奈': 24, '谷本 心': 25, 'リブロワークス': 26, '佐々木整': 27, '石井 真': 28, '株式会社カサレアル': 29, 'きしだなおき': 30, '山本裕介': 31, '杉山貴章': 32, '三谷 純': 33, '近藤 雄太': 34, '正野 裕大': 35, '坂井 潔': 36, '鳥越 淳': 37, '笠原 辰仁': 38, '勝俣智成': 39, '佐伯昌樹': 40, '原田登志': 41, '西潤史郎': 42, 'ミック': 43, '常松 祐一': 44, '安達裕哉': 45, 'ウォルター・アイザックソン': 46, 'Mana': 47, '佐竹陽一': 48, '山﨑翔平': 49, '小倉大': 50, '峯侑資': 51, '馬田隆明': 52, '山田祥寛': 53, '西岡杏': 54, '増井 敏克': 55, 'IPUSIRON': 56, 'トム・ロング (ソフトウェア工学)': 57, '高松智史': 58, '木下勝寿': 59, 'アレックス・シュウ': 60, '神林飛志': 61, '荒川傑': 62, '菱田真人': 63, 'シド・ハレル': 64, 'azu': 65, 'Suguru Inatomi': 66, '荒木俊哉': 67, 'アンドリュー・スチュワート': 68, 'ピョートル・フェリクス・グジバチ': 69, '木暮太一': 70, 'Will Larson': 71, 'フレアリンク': 72, '牛尾剛': 73, 

In [33]:
embedding_dim = 50
author_embedding = nn.Embedding(num_embeddings=len(author2id), embedding_dim=embedding_dim)
torch.save(author_embedding.state_dict(), 'author_embedding.pth')

In [35]:
test_name = '今井雄太'
test_id = author2id.get(test_name)
test_vector = author_embedding(torch.tensor(test_id))
print(test_vector)

tensor([-1.1671, -1.3120, -0.5438,  0.1634, -0.1533, -1.7294,  0.1975,  0.9962,
        -0.3044, -0.6423,  0.7495,  0.5213,  0.4483,  0.8157,  0.1348, -0.3277,
        -1.7318,  0.0417, -0.2170,  0.3519, -0.7005,  0.3271, -0.7712, -0.3103,
        -0.5458,  0.4197,  0.2136, -1.7106, -0.3885,  0.5985, -0.0180, -1.3629,
         0.5803, -0.6258,  0.1232,  0.7960,  0.5229,  0.7541,  0.8065,  0.2974,
        -0.1627, -0.8344,  2.0857,  2.0902, -0.5815,  0.9906, -0.1330,  0.2622,
         0.6056,  0.9648], grad_fn=<EmbeddingBackward0>)


In [37]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

H:\anaconda\envs\BoxOffice\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liufe\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

H:\anaconda\envs\BoxOffice\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [39]:
def embedding_title_description(title, description):
    encoded_input = tokenizer(title, description, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
    return output.last_hidden_state[:, 0, :]

In [42]:
test_book_1 = data_df.loc[0]
print(test_book_1)

isbn                                          9784297106614
title                         ゼロからわかるAmazon Web Services超入門
authors                                              [大澤文孝]
description    AWSでWebサイトを構築・運用!実際に動かして学べる1冊。練習問題&解答・解説集付き。
categories                                        [Unknown]
Name: 0, dtype: object
tensor([[ 1.2842e-01,  6.4938e-02,  4.1130e-02, -7.7351e-02, -4.0610e-02,
          3.3714e-02,  1.8975e-01,  8.0770e-02, -5.5414e-02,  3.4873e-02,
          1.1634e-01,  1.4467e-01, -9.3241e-02, -1.1470e-02, -6.6672e-01,
         -1.0164e-01, -1.1545e-01,  1.3759e-01,  9.5735e-02,  1.9998e-01,
          2.0766e-01,  9.0271e-02, -5.4851e-02,  6.6033e-03, -6.8915e-02,
         -1.4335e-01, -2.0523e-02,  9.1148e-02,  4.1983e-01,  8.4142e-02,
          9.6159e-02,  5.1262e-02, -7.7904e-02, -5.1419e-02,  1.0739e-01,
         -1.0456e-01, -2.2085e+00, -1.1411e-01, -9.3080e-03, -3.0114e-02,
          2.1611e-02, -4.6261e-02, -1.5829e-01,  6.1543e-02,  2.9338e-02,
          1

In [43]:
test_embedded_text = embedding_title_description(test_book_1.get("title"), test_book_1.get("description"))

In [45]:
print(test_embedded_text.shape)
print(test_embedded_text)

torch.Size([1, 768])
tensor([[ 1.2842e-01,  6.4938e-02,  4.1130e-02, -7.7351e-02, -4.0610e-02,
          3.3714e-02,  1.8975e-01,  8.0770e-02, -5.5414e-02,  3.4873e-02,
          1.1634e-01,  1.4467e-01, -9.3241e-02, -1.1470e-02, -6.6672e-01,
         -1.0164e-01, -1.1545e-01,  1.3759e-01,  9.5735e-02,  1.9998e-01,
          2.0766e-01,  9.0271e-02, -5.4851e-02,  6.6033e-03, -6.8915e-02,
         -1.4335e-01, -2.0523e-02,  9.1148e-02,  4.1983e-01,  8.4142e-02,
          9.6159e-02,  5.1262e-02, -7.7904e-02, -5.1419e-02,  1.0739e-01,
         -1.0456e-01, -2.2085e+00, -1.1411e-01, -9.3080e-03, -3.0114e-02,
          2.1611e-02, -4.6261e-02, -1.5829e-01,  6.1543e-02,  2.9338e-02,
          1.3523e+00,  1.1570e-01, -1.0386e-01,  1.6415e+00,  6.7092e-02,
          1.3935e-01, -7.5360e-01,  4.1243e-02, -1.7819e+00,  9.5956e-02,
          1.3021e-01,  1.0459e-01, -5.8038e-02, -8.5879e-02,  1.3866e-01,
          1.2355e-01,  1.0099e-02, -8.7949e-03, -1.3938e-01, -9.6190e-02,
          7.2328e

In [52]:
data_df = data_df.explode('categories')
data_df = pd.get_dummies(data_df, columns=['categories'])

In [53]:
data_df

,isbn,title,authors,description,categories_Business & Economics,categories_Computers,categories_Language Arts & Disciplines,categories_Reference,categories_Self-Help,categories_Technology & Engineering,categories_Unknown
0,9784297106614,ゼロからわかるAmazon Web Services超入門,[大澤文孝],AWSでWebサイトを構築・運用!実際に動かして学べる1冊。練習問題&解答・解説集付き。,False,False,False,False,False,False,True
1,9784297108892,図解即戦力 Amazon Web Servicesのしくみと技術がこれ1冊でしっかりわかる教科書,[小笠原種高],Amazon Web Servicesのしくみがキーワードベースでわかる!ネットワーク&クラ...,False,False,False,False,False,False,True
2,9784798162560,AWSクラウドの基本と仕組み,"[亀田 治伸, 山田 裕進]",AWS入門書の決定版 アマゾン ウェブ サービス(AWS)社のプロダクトマーケティング エバ...,False,False,False,False,False,False,True
3,9784296202041,Amazon Web Services基礎からのネットワーク&サーバー構築改訂4版,"[大澤文孝, 玉川憲, 片山暁雄, 今井雄太]",AWSを使ってインフラの基本を学べる本。「クラウドを触って学ぶ」コンセプトが広く受け入れられ...,False,False,False,False,False,False,True
4,9784797392562,Amazon Web Services 業務システム設計・移行ガイド,"[佐々木拓郎, 林晋一郎, 瀬戸島敏宏, 宮川亮, 金澤圭]",社内システムのクラウド化を目指す人に！ ※この電子書籍は固定レイアウト型で配信されております...,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
95,9784863543324,Vue.jsビギナーズガイド 3.x対応,[ushironoko],Vue.js3.0の入門に最適!,False,False,False,False,False,False,True
96,9784863543522,現場で使えるVue.js 3.X 実践ガイド,[MONSTER DIVE],Vue.jsの実践が学べる1冊!,False,False,False,False,False,False,True
97,9784297130725,Vue 3 フロントエンド開発の教科書,[WINGSプロジェクト齊藤新三],人気のWebフロントエンドフレームワーク「Vue.js」がバージョンアップして「Vue 3」...,False,False,False,False,False,False,True
98,9784798158921,動かして学ぶ！Vue.js開発入門,[森巧尚],【Vue.js（ビュージェイエス）とは】 Vue.jsは、Webアプリ開発用のJavaScr...,False,False,False,False,False,False,True


In [95]:
def get_author_vector(name_list):
    author_vectors = []
    for name in name_list:
        name_id = torch.tensor(author2id.get(name))
        author_vectors.append(author_embedding(name_id))
    # print(len(author_vectors))
    return torch.mean(torch.stack(author_vectors), dim=0)



In [99]:
categories_list = [col for col in data_df.columns if 'categories' in col]
training_data = []
for index, row in data_df.iterrows():
    temp_data = {
        'isbn': row['isbn'],
        'tensor': []
    }
    categories_values = row[categories_list].values.astype(int).tolist()
    temp_title_description = embedding_title_description(row['title'], row['description'])
    temp_data_author = get_author_vector(row['authors'])
    temp_data_categories = torch.tensor(categories_values)
    flatten_data = torch.cat((temp_title_description.flatten(), temp_data_author.flatten(), temp_data_categories.flatten()), dim=0)
    temp_data['tensor'] = flatten_data
    training_data.append(temp_data)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [100]:
print(training_data[0])
print(training_data[0].get('tensor').shape)

{'isbn': 9784297106614, 'tensor': tensor([ 1.2842e-01,  6.4938e-02,  4.1130e-02, -7.7351e-02, -4.0610e-02,
         3.3714e-02,  1.8975e-01,  8.0770e-02, -5.5414e-02,  3.4873e-02,
         1.1634e-01,  1.4467e-01, -9.3241e-02, -1.1470e-02, -6.6672e-01,
        -1.0164e-01, -1.1545e-01,  1.3759e-01,  9.5735e-02,  1.9998e-01,
         2.0766e-01,  9.0271e-02, -5.4851e-02,  6.6033e-03, -6.8915e-02,
        -1.4335e-01, -2.0523e-02,  9.1148e-02,  4.1983e-01,  8.4142e-02,
         9.6159e-02,  5.1262e-02, -7.7904e-02, -5.1419e-02,  1.0739e-01,
        -1.0456e-01, -2.2085e+00, -1.1411e-01, -9.3080e-03, -3.0114e-02,
         2.1611e-02, -4.6261e-02, -1.5829e-01,  6.1543e-02,  2.9338e-02,
         1.3523e+00,  1.1570e-01, -1.0386e-01,  1.6415e+00,  6.7092e-02,
         1.3935e-01, -7.5360e-01,  4.1243e-02, -1.7819e+00,  9.5956e-02,
         1.3021e-01,  1.0459e-01, -5.8038e-02, -8.5879e-02,  1.3866e-01,
         1.2355e-01,  1.0099e-02, -8.7949e-03, -1.3938e-01, -9.6190e-02,
         7.2328e-

In [109]:
training_data_json = [{'isbn': data.get('isbn'), 'tensor':data.get('tensor').tolist()} for data in training_data]
training_data_json[0]
with open('book_tensor_data.json', 'w') as json_file:
    json.dump(training_data_json, json_file)

In [105]:
training_data[0].get('tensor')

tensor([ 1.2842e-01,  6.4938e-02,  4.1130e-02, -7.7351e-02, -4.0610e-02,
         3.3714e-02,  1.8975e-01,  8.0770e-02, -5.5414e-02,  3.4873e-02,
         1.1634e-01,  1.4467e-01, -9.3241e-02, -1.1470e-02, -6.6672e-01,
        -1.0164e-01, -1.1545e-01,  1.3759e-01,  9.5735e-02,  1.9998e-01,
         2.0766e-01,  9.0271e-02, -5.4851e-02,  6.6033e-03, -6.8915e-02,
        -1.4335e-01, -2.0523e-02,  9.1148e-02,  4.1983e-01,  8.4142e-02,
         9.6159e-02,  5.1262e-02, -7.7904e-02, -5.1419e-02,  1.0739e-01,
        -1.0456e-01, -2.2085e+00, -1.1411e-01, -9.3080e-03, -3.0114e-02,
         2.1611e-02, -4.6261e-02, -1.5829e-01,  6.1543e-02,  2.9338e-02,
         1.3523e+00,  1.1570e-01, -1.0386e-01,  1.6415e+00,  6.7092e-02,
         1.3935e-01, -7.5360e-01,  4.1243e-02, -1.7819e+00,  9.5956e-02,
         1.3021e-01,  1.0459e-01, -5.8038e-02, -8.5879e-02,  1.3866e-01,
         1.2355e-01,  1.0099e-02, -8.7949e-03, -1.3938e-01, -9.6190e-02,
         7.2328e-02,  8.4754e-02, -1.3094e-01, -7.3